# 🌐 Distributed Systems Patterns

**Phase 3: System Design - Scalable Distributed Architecture**

**Master CAP theorem, consistency models, partitioning, and distributed fault tolerance**

---

In [ ]:
// Distributed Systems Fundamentals
println("🌐 DISTRIBUTED SYSTEMS - SCALABILITY & CONSISTENCY MASTERCLASS")
println()

println("🔥 The Eight Fallacies of Distributed Computing:")
println("1. ❌ The network is reliable")
println("2. ❌ Latency is zero")
println("3. ❌ Bandwidth is infinite")
println("4. ❌ The network is secure")
println("5. ❌ Topology doesn't change")
println("6. ❌ There is one administrator")
println("7. ❌ Transport cost is zero")
println("8. ❌ The network is homogeneous")
println()

println("🎯 CAP Theorem Trade-offs:")
println("• Consistency: All nodes see same data simultaneously")
println("• Availability: Every request receives a response")
println("• Partition Tolerance: System works despite network splits")
println("• Choose 2 out of 3 in the face of network partitions")
println()

println("⚡ PACELC Theorem (extension):")
println("• If Partition (P), choose Availability (A) or Consistency (C)")
println("• Else when Latency (L), choose between low Latency (L) or high Consistency (C)")


## 📊 Consistency Models

**Serializability vs. Eventual Consistency vs. Strong Consistency - choosing the right model**

In [ ]:
// Consistency Models Implementation
sealed trait ConsistencyLevel
case object StrongConsistency extends ConsistencyLevel      // Linearizability/Serializability
case object EventualConsistency extends ConsistencyLevel   // BASE properties
case object ReadYourWrites extends ConsistencyLevel         // Session consistency
case object MonotonicReads extends ConsistencyLevel         // No time travel
case object CausalConsistency extends ConsistencyLevel     // Causally related writes
case object BoundedStaleness extends ConsistencyLevel      // Maximum staleness guarantee

case class ConsistencyRequirements(
  system: String,
  readLoad: Double,
  writeLoad: Double,
  acceptableLatency: Long,
  consistencyNeeded: ConsistencyLevel,
  availabilityTarget: Double
)

object ConsistencyAdvisor {
  
  def recommendCoherenceProtocol(reqs: ConsistencyRequirements): String = {
    val loadRatio = reqs.readLoad / math.max(reqs.writeLoad, 1)
    val highReadLowWrite = loadRatio > 10
    val strictConsistency = reqs.consistencyNeeded == StrongConsistency
    
    // Recommendation logic
    (reqs, loadRatio, strictConsistency) match {
      case (reqs, lr, true) if lr > 100 => 
        "Quorum-based consensus (Paxos/Raft) with read-modify-write cycles"
      case (reqs, lr, true) if lr > 10 => 
        "Two-phase commit or synchronous replication"
      case (reqs, lr, true) => 
        "Leader-based replication with WAL persistence"
      case (reqs, lr, false) if reqs.acceptableLatency < 50 => 
        "Eventual consistency with anti-entropy (Amazon Dynamo-style)"
      case (reqs, lr, false) if reqs.acceptableLatency < 200 => 
        "Cassandra-style tunable consistency levels"
      case _ => "Multi-region active-active with CRDTs"
    }
  }
  
  def tradeOffAnalysis(reqs: ConsistencyRequirements): Map[String, String] = Map(
    "performance" -> {
      reqs.consistencyNeeded match {
        case StrongConsistency => "Slower due to coordination overhead"
        case EventualConsistency => "Fast reads, coordination on conflicts"
        case _ => "Balanced with tunable trade-offs"
      }
    },
    "fault_tolerance" -> {
      reqs.availabilityTarget match {
        case x if x >= 0.9999 => "High replication, consistency relaxations"
        case x if x >= 0.99 => "Multi-region, quorum-based decisions"
        case _ => "Synchronous replication, stricter consistency"
      }
    },
    "complexity" -> {
      if (reqs.consistencyNeeded == StrongConsistency) "High - distributed locks, coordination"
      else if (reqs.consistencyNeeded == EventualConsistency) "Medium - conflict resolution strategies"
      else "High - custom consistency protocols"
    }
  )
}

println("📊 CONSISTENCY MODEL ANALYSIS IMPLEMENTED")
println("• Strong Consistency: Sequential operations, coordination overhead")
println("• Eventual Consistency: Conflicts require resolution, higher throughput")
println("• Read-Your-Writes: Session guarantees, UI-friendly")
println("• Causal Consistency: Partial ordering, dependency tracking")
println("• Choosing right model: Application requirements drive selection")
println()


## 🗂️ Data Partitioning Strategies

**Horizontal partitioning, sharding, and data distribution patterns**

In [ ]:
// Data Partitioning Strategies
sealed trait PartitioningStrategy
case object HashPartitioning extends PartitioningStrategy        // Even distribution, range queries hard
case object RangePartitioning extends PartitioningStrategy       // Range queries easy, uneven distribution
case object AdaptivePartitioning extends PartitioningStrategy    // Hybrid approach
case object DirectoryPartitioning extends PartitioningStrategy   // Lookup table, high lookup overhead
case object EntityPartitioning extends PartitioningStrategy      // Domain-driven partitioning

case class PartitionInfo(
  node: String,
  minKey: String,
  maxKey: String,
  recordCount: Long,
  sizeInMB: Double
)

class PartitionManager {
  
  private val partitions = scala.collection.mutable.Map[String, PartitionInfo]()
  
  // Hash-based partitioning
  def hashPartition(key: String, numPartitions: Int): Int = {
    val hash = key.hashCode.abs
    hash % numPartitions
  }
  
  // Range-based partitioning (for sorted keys)
  def rangePartition(key: String, ranges: Map[Int, (String, String)]): Option[Int] = {
    ranges.collectFirst {
      case (partition, (min, max)) if key >= min && key <= max => partition
    }
  }
  
  // Consistency routing
  def findPartition(key: String, strategy: PartitioningStrategy): String = {
    strategy match {
      case HashPartitioning => 
        val partitionId = hashPartition(key, partitions.size)
        s"node_$partitionId"
      case RangePartitioning =>
        // Find partition containing key
        partitions.find(_._2.minKey <= key && _._2.maxKey >= key)
          .map(_._1).getOrElse("unknown")
      case _ => "adaptive" // Simplified for demo
    }
  }
  
  // Rebalancing when adding/removing nodes
  def rebalancePartitions(clusterSize: Int): Map[String, (Long, Long)] = {
    // Calculate key ranges per partition
    val totalKeys = math.pow(2, 32).toLong // Assume 32-bit keys
    val keysPerNode = totalKeys / clusterSize
    
    (0 until clusterSize).map { nodeId =>
      val startKey = nodeId * keysPerNode
      val endKey = if (nodeId == clusterSize - 1) totalKeys - 1 else (nodeId + 1) * keysPerNode - 1
      s"node_$nodeId" -> (startKey, endKey)
    }.toMap
  }

  def getPartitionMetrics(): Map[String, PartitionInfo] = partitions.toMap
  def addPartition(node: String, info: PartitionInfo): Unit = partitions.put(node, info)
}

println("🗂️ DATA PARTITIONING STRATEGIES IMPLEMENTED")
println("• Hash Partitioning: Even load, range queries require scatter-gather")
println("• Range Partitioning: Range queries fast, hotspots possible")
println("• Adaptive Partitioning: Hybrid strategies for changing workloads")
println("• Entity Partitioning: Domain boundaries, reduces cross-entity queries")
println()


## 🔄 Consensus & Coordination

**Leader election, distributed locks, and agreement protocols**

In [ ]:
// Consensus Protocols Implementation
sealed trait ConsensusProtocol
case object Paxos extends ConsensusProtocol          // Distributed consensus
case object Raft extends ConsensusProtocol           // Leader election + replication
case object ViewStampedReplication extends ConsensusProtocol
case object ZAB extends ConsensusProtocol           // ZooKeeper's protocol

sealed trait ConsensusState
case class Leader(term: Long, nodeId: String) extends ConsensusState
case class Follower(term: Long, leaderId: String) extends ConsensusState
case class Candidate(term: Long, votes: Set[String]) extends ConsensusState

case class ConsensusMessage(
  msgType: String,      // "RequestVote", "AppendEntries", etc.
  term: Long,
  sender: String,
  payload: Map[String, Any],
  timestamp: java.time.Instant = java.time.Instant.now()
)

class SimplifiedRaft {
  private var state: ConsensusState = Follower(0L, "")
  private val log = scala.collection.mutable.ArrayBuffer[(Long, Any)]()
  private val peers = Set("node_1", "node_2", "node_3")
  
  // Election timeout (simplified)
  def startElection(): ConsensusState = {
    val currentTerm = state match {
      case Follower(term, _) => term + 1
      case Candidate(term, _) => term + 1
      case Leader(term, _) => term
    }
    
    state = Candidate(currentTerm, Set.empty)
    val votesReceived = peers.take(scala.util.Random.nextInt(peers.size)) // Random vote success
    
    if (votesReceived.size >= (peers.size + 1) / 2) { // Majority vote
      state = Leader(currentTerm, "self")
      println(s"🎯 Elected leader for term $currentTerm")
    } else {
      println(s"❌ Election failed for term $currentTerm")
      state = Follower(currentTerm, "")
    }
    state
  }
  
  // Heartbeat/AppendEntries (simplified)
  def sendHeartbeat(leaderTerm: Long): Unit = {
    peers.foreach { peer =>
      val message = ConsensusMessage(
        "AppendEntries",
        leaderTerm,
        "self",
        Map("leaderCommit" -> log.size, "entries" -> List.empty[Any])
      )
      println(s"💓 Heartbeat to $peer in term $leaderTerm")
    }
  }
  
  def appendEntries(term: Long, entries: List[(Long, Any)]): Either[String, Unit] = {
    val currentTerm = state match {
      case Leader(t, _) if t == term || t < term => t
      case Follower(t, _) if t <= term => term
      case _ => 0L
    }
    
    log.appendAll(entries)
    Right(())  // In reality, would check sequence numbers
  }
  
  def getLog() = log.toList
  def getState() = state
}

println("🔄 CONSENSUS & COORDINATION IMPLEMENTED")
println("• Leader election: Majority voting, term advancement")
println("• Log replication: Append-only logs with consistency checks")
println("• Heartbeat monitoring: Leader failure detection")
println("• Term management: Election cycles and conflict resolution")
println()


## 🏗️ Distributed System Architectures

**Lambda/Kappa architectures, mastering batch vs stream processing**

In [ ]:
// Lambda Architecture Implementation
sealed trait DataLayer
case object BatchLayer extends DataLayer         // Hadoop, compute views on all data
case object SpeedLayer extends DataLayer         // Streaming, approximate results
case object ServingLayer extends DataLayer       // Pre-computed views, low-latency access

// Kappa Architecture (streaming-first)
sealed trait KappaComponent
case object StreamProcessor extends KappaComponent
case object StateStore extends KappaComponent
case object SlowLayer extends KappaComponent     // Reprocessing stream from beginning

case class DataPipeline(
  architecture: String,  // "lambda" or "kappa"
  batchLayer: Option[DataLayer],
  speedLayer: Option[DataLayer],
  servingLayer: DataLayer,
  streamProcessor: Option[KappaComponent]
)

object ArchitectureComparison {
  
  val lambdaArchitecture = DataPipeline(
    "lambda",
    Some(BatchLayer),
    Some(SpeedLayer),
    ServingLayer,
    None
  )
  
  val kappaArchitecture = DataPipeline(
    "kappa",
    None,
    None,
    ServingLayer,
    Some(StreamProcessor)
  )
  
  def compareArchitectures(): Map[String, String] = {
    Map(
      "complexity" -> "Lambda: Two code paths, higher complexity. Kappa: Single stream processing pipeline",
      "consistency" -> "Lambda: Eventual consistency between layers. Kappa: Eventually consistent single system",
      "real_time" -> "Lambda: Faster, two-tier processing. Kappa: Single pipeline, potentially slower",
      "reprocessing" -> "Lambda: Reprocess batch data. Kappa: Reset stream from beginning",
      "development" -> "Lambda: Maintain batch + streaming code. Kappa: Unified stream processing",
      "scalability" -> "Both scale, Kappa often simpler for stream-first use cases",
      "use_case" -> "Lambda: When batch quality > speed. Kappa: When speed = quality priority"
    )
  }
  
  def chooseArchitecture(requirements: Map[String, Any]): String = {
    val latencyCritical = requirements.get("latency_ms").map(_.asInstanceOf[Long] < 100).getOrElse(false)
    val reprocessingFreq = requirements.get("reprocess_frequency").map(_.asInstanceOf[String]).getOrElse("rare")
    val teamSize = requirements.get("team_size").map(_.asInstanceOf[Int]).getOrElse(10)
    
    (latencyCritical, reprocessingFreq, teamSize) match {
      case (true, freq, size) if size > 5 => "kappa"  // Stream-first, team can handle complexity
      case (false, "frequent", _) => "lambda"         // Batch reprocessing, different latency needs
      case (false, "rare", size) if size <= 5 => "kappa" // Simpler pipeline for smaller team
      case _ => "lambda"                            // Default to battle-tested lambda
    }
  }
  
  def getAdvantages(arch: String): List[String] = {
    arch.toLowerCase match {
      case "lambda" => List(
        "Battle-tested architecture",
        "Separates accuracy and speed concerns",
        "Batch layer handles data corrections",
        "Speed layer serves low-latency needs",
        "Well-established patterns (Rolling, Pinball, etc.)"
      )
      case "kappa" => List(
        "Single pipeline - simpler operations",
        "Unified stream processing code",
        "Better real-time capabilities",
        "No serving layer consistency issues",
        "Easier testing with deterministic reprocessing"
      )
      case _ => List("Unknown architecture")
    }
  }
}

println("🏗️ DISTRIBUTED SYSTEM ARCHITECTURES: LAMBDA vs KAPPA")
println()
println("🔥 Key Decision Factors:")
println("✓ Team expertise and size")
println("✓ Data reprocessing frequency")
println("✓ Latency requirements")
println("✓ Operational complexity tolerance")
println("✓ Historical data correction importance")
println()

println("💡 Architecture Evolution:")
println("2010s: Lambda (batch + stream)")
println("2020s: Kappa (stream-first, faster processing)")
println("Future: Zeta (AI-powered, self-optimizing)")
println()

println("🎯 Choose based on your specific use case, not trends!")
